In [1]:
import numpy as np
import pandas as pd
from ahrs.filters import Complementary

def complementary_filter(df):
    # inicialização do filtro complementar, gain -> 1 giroscópio, gain -> 0 acelerômetro
    cf = Complementary(frequency=100, gain=0.75)
    
    # grupo dos sensores
    sensor_groups = {
        'right_foot': ['accelerometer_right_foot_x', 'accelerometer_right_foot_y', 'accelerometer_right_foot_z', 'gyroscope_right_foot_x', 'gyroscope_right_foot_y', 'gyroscope_right_foot_z'],
        'right_shin': ['accelerometer_right_shin_x', 'accelerometer_right_shin_y', 'accelerometer_right_shin_z', 'gyroscope_right_shin_x', 'gyroscope_right_shin_y', 'gyroscope_right_shin_z'],
        'right_thigh': ['accelerometer_right_thigh_x', 'accelerometer_right_thigh_y', 'accelerometer_right_thigh_z', 'gyroscope_right_thigh_x', 'gyroscope_right_thigh_y', 'gyroscope_right_thigh_z'],
        'left_foot': ['accelerometer_left_foot_x', 'accelerometer_left_foot_y', 'accelerometer_left_foot_z', 'gyroscope_left_foot_x', 'gyroscope_left_foot_y', 'gyroscope_left_foot_z'],
        'left_shin': ['accelerometer_left_shin_x', 'accelerometer_left_shin_y', 'accelerometer_left_shin_z', 'gyroscope_left_shin_x', 'gyroscope_left_shin_y', 'gyroscope_left_shin_z'],
        'left_thigh': ['accelerometer_left_thigh_x', 'accelerometer_left_thigh_y', 'accelerometer_left_thigh_z', 'gyroscope_left_thigh_x', 'gyroscope_left_thigh_y', 'gyroscope_left_thigh_z']
    }
    
    orientations_dict = {sensor: [] for sensor in sensor_groups}

    # orientação inicial do quaternion, assumindo que não há rotação no início da leitura
    orientation = np.array([1.0, 0.0, 0.0, 0.0])

    # iteração entre todos os membros do df
    for i in range(len(df)):
        for sensor, columns in sensor_groups.items():
            # dados do acelerômetro e giroscópio, iterando sobre todos os grupos de sensores
            acc = np.array([df.iloc[i][columns[0]], df.iloc[i][columns[1]], df.iloc[i][columns[2]]])
            gyro = np.array([df.iloc[i][columns[3]], df.iloc[i][columns[4]], df.iloc[i][columns[5]]])
            
            # atualização do quaternion, utilizando o quaternion anterior
            orientation = cf.update(orientation, gyr=gyro, acc=acc)
            orientations_dict[sensor].append(orientation)

    # criação do df filtrado
    orientation_dfs = []
    for sensor, orientations in orientations_dict.items():
        orientation_df = pd.DataFrame(orientations, columns=[f'{sensor}_q0', f'{sensor}_q1', f'{sensor}_q2', f'{sensor}_q3'])
        orientation_dfs.append(orientation_df)

    return pd.concat(orientation_dfs, axis=1)

df = pd.read_csv('./csv/database.csv')

df_filtered = complementary_filter(df)
df_filtered['activity'] = df['activity'].values

# Conversão do df em um único csv
df_filtered.to_csv('./csv/database_complementary_filter.csv', index=False)


In [3]:
display(df_filtered.describe())

display(df_filtered.head())

print(df_filtered.info())

,right_foot_q0,right_foot_q1,right_foot_q2,right_foot_q3,right_shin_q0,right_shin_q1,right_shin_q2,right_shin_q3,right_thigh_q0,right_thigh_q1,...,left_foot_q2,left_foot_q3,left_shin_q0,left_shin_q1,left_shin_q2,left_shin_q3,left_thigh_q0,left_thigh_q1,left_thigh_q2,left_thigh_q3
count,361344.000000,361344.000000,361344.000000,361344.000000,361344.000000,361344.000000,361344.000000,361344.000000,361344.000000,361344.000000,...,361344.000000,361344.000000,361344.000000,361344.000000,361344.000000,361344.000000,361344.000000,361344.000000,361344.000000,361344.000000
mean,0.854534,-0.146477,0.270742,0.026427,0.395851,-0.210923,0.235134,0.086455,0.450011,0.073076,...,0.257702,-0.052141,0.349169,0.143786,0.216172,-0.089548,0.468905,0.049798,0.271089,-0.014060
std,0.154386,0.278707,0.185152,0.196272,0.229517,0.669202,0.231948,0.426313,0.304645,0.621335,...,0.188663,0.200884,0.228037,0.700412,0.245930,0.446767,0.302207,0.611044,0.216796,0.438473
min,-0.320511,-0.994123,-0.744109,-0.873057,-0.317902,-0.999995,-0.803547,-0.887152,-0.330311,-0.999934,...,-0.758820,-0.868583,-0.324369,-0.999883,-0.807511,-0.854764,-0.327836,-0.999906,-0.862628,-0.891833
25%,0.840206,-0.333975,0.211930,-0.103724,0.225931,-0.750846,0.121258,-0.411433,0.217343,-0.515324,...,0.193673,-0.207363,0.188905,-0.656735,0.083837,-0.490718,0.270378,-0.593519,0.136679,-0.394603
50%,0.895607,-0.167843,0.298062,0.038293,0.415652,-0.568046,0.274658,0.267054,0.402355,0.077681,...,0.284457,-0.082955,0.335105,0.520882,0.229157,-0.290230,0.425167,0.078185,0.234971,-0.041811
75%,0.940386,-0.028399,0.360569,0.172772,0.542722,0.573737,0.366243,0.461554,0.711270,0.711158,...,0.362143,0.079044,0.488840,0.751513,0.399875,0.359405,0.736335,0.683464,0.431045,0.391535
max,0.999967,0.996836,0.871828,0.882415,0.999161,0.999749,0.876084,0.867366,0.999787,0.999933,...,0.865737,0.881163,0.999810,0.999952,0.879775,0.874487,0.999846,0.999759,0.865222,0.890488


,right_foot_q0,right_foot_q1,right_foot_q2,right_foot_q3,right_shin_q0,right_shin_q1,right_shin_q2,right_shin_q3,right_thigh_q0,right_thigh_q1,...,left_foot_q3,left_shin_q0,left_shin_q1,left_shin_q2,left_shin_q3,left_thigh_q0,left_thigh_q1,left_thigh_q2,left_thigh_q3,activity
0,0.970829,-0.070381,0.228369,-0.019610,0.317460,-0.825455,0.046582,0.464406,0.917718,-0.288387,...,-0.017643,0.331640,0.803154,0.081904,-0.488109,0.931488,0.298828,0.140535,-0.152581,sitting
1,0.959722,0.014950,0.277810,-0.039130,0.321908,-0.819228,0.070754,0.469291,0.919923,-0.292079,...,-0.016717,0.336851,0.799394,0.080728,-0.490901,0.931984,0.296338,0.141919,-0.153128,sitting
2,0.960639,0.016235,0.273444,-0.046232,0.319796,-0.816293,0.060749,0.477186,0.921759,-0.282496,...,-0.010836,0.339921,0.794227,0.083226,-0.496720,0.933935,0.290860,0.135102,-0.157842,sitting
3,0.962366,0.018291,0.267725,-0.042914,0.313665,-0.821108,0.055391,0.473632,0.922880,-0.279656,...,-0.014164,0.345871,0.795016,0.083422,-0.491287,0.931814,0.294269,0.145796,-0.154504,sitting
4,0.960306,0.022584,0.274994,-0.041008,0.323863,-0.814983,0.070604,0.475322,0.924312,-0.271967,...,-0.010293,0.339269,0.800559,0.086134,-0.486398,0.933830,0.288712,0.139876,-0.158245,sitting


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361344 entries, 0 to 361343
Data columns (total 25 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   right_foot_q0   361344 non-null  float64
 1   right_foot_q1   361344 non-null  float64
 2   right_foot_q2   361344 non-null  float64
 3   right_foot_q3   361344 non-null  float64
 4   right_shin_q0   361344 non-null  float64
 5   right_shin_q1   361344 non-null  float64
 6   right_shin_q2   361344 non-null  float64
 7   right_shin_q3   361344 non-null  float64
 8   right_thigh_q0  361344 non-null  float64
 9   right_thigh_q1  361344 non-null  float64
 10  right_thigh_q2  361344 non-null  float64
 11  right_thigh_q3  361344 non-null  float64
 12  left_foot_q0    361344 non-null  float64
 13  left_foot_q1    361344 non-null  float64
 14  left_foot_q2    361344 non-null  float64
 15  left_foot_q3    361344 non-null  float64
 16  left_shin_q0    361344 non-null  float64
 17  left_shin_